In [1]:
import os
import cv2
import numpy as np

def load_images_to_numpy(folder_path, size=(60, 40)):
    """
    Загружает изображения из папки, изменяет их размер и конвертирует в массивы NumPy.

    :param folder_path: Путь к папке с изображениями.
    :param size: Новый размер изображений (ширина, высота).
    :return: Массив NumPy, содержащий изображения.
    """
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = cv2.imread(img_path)
            img_resized = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
            images.append(img_resized)
    return np.array(images, dtype='uint8')

# Загрузка изображений из каждой папки
proliv_images = load_images_to_numpy('dataset/proliv')
test_images = load_images_to_numpy('dataset/test/imgs')
train_images = load_images_to_numpy('dataset/train')

# Вывод информации о загруженных массивах
print(f'Проливы: {proliv_images.shape}')
print(f'Тестовые изображения: {test_images.shape}')
print(f'Обучающие изображения: {train_images.shape}')


Проливы: (154, 40, 60, 3)
Тестовые изображения: (3794, 40, 60, 3)
Обучающие изображения: (10000, 40, 60, 3)


In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

# Функция load_images_to_numpy из предыдущего примера

class Encoder(nn.Module):
    def __init__(self, latent_dims):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(60 * 40 * 3, 512)  # Изменен размер входа
        self.linear2 = nn.Linear(512, latent_dims)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)  # Выпрямление изображения в одномерный вектор
        x = F.relu(self.linear1(x))
        return self.linear2(x)

class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 60 * 40 * 3)  # Изменен размер выхода

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return z.view(-1, 3, 40, 60)  # Изменение формы выхода для соответствия размеру изображения

class Autoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

import tqdm

import tqdm

def train(autoencoder, data_loader, epochs=100, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    optimizer = torch.optim.Adam(autoencoder.parameters())
    criterion = nn.MSELoss()

    for epoch in tqdm.tqdm(range(epochs)):
        total_loss = 0  # Суммарная потеря за эпоху
        num_batches = 0  # Количество батчей

        for img_batch in data_loader:
            img = img_batch[0].to(device)  # Перенос на GPU, если доступно
            optimizer.zero_grad()
            reconstructed = autoencoder(img)
            loss = criterion(reconstructed, img)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()  # Добавление потери от батча к общей сумме
            num_batches += 1

        average_loss = total_loss / num_batches  # Средняя потеря за эпоху
        print(f"Epoch {epoch+1}/{epochs}, Average Loss: {average_loss:.4f}")

    return autoencoder




# Загрузка и подготовка данных
proliv_images = load_images_to_numpy('dataset/proliv')  # Пример использования функции
train_images = load_images_to_numpy('dataset/train')  # Пример использования функции

# Преобразование массивов NumPy в тензоры PyTorch
train_images_tensor = torch.tensor(train_images, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0  # Преобразование и нормализация

# Создание DataLoader
train_loader = DataLoader(TensorDataset(train_images_tensor), batch_size=64, shuffle=True)

# Инициализация и обучение автоэнкодера
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
latent_dims = 1024  # Можно настроить
autoencoder = Autoencoder(latent_dims).to(device)
train(autoencoder, train_loader, epochs=100)


  1%|▊                                                                                 | 1/100 [00:00<00:50,  1.95it/s]

Epoch 1/100, Average Loss: 0.0017


  2%|█▋                                                                                | 2/100 [00:01<00:49,  1.99it/s]

Epoch 2/100, Average Loss: 0.0014


  3%|██▍                                                                               | 3/100 [00:01<00:48,  1.98it/s]

...

 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:50<00:00,  1.97it/s]

Epoch 99/100, Average Loss: 0.0010


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:50<00:00,  1.98it/s]

Epoch 100/100, Average Loss: 0.0011


Autoencoder(
  (encoder): Encoder(
    (linear1): Linear(in_features=7200, out_features=512, bias=True)
    (linear2): Linear(in_features=512, out_features=1024, bias=True)
  )
  (decoder): Decoder(
    (linear1): Linear(in_features=1024, out_features=512, bias=True)
    (linear2): Linear(in_features=512, out_features=7200, bias=True)
  )
)

In [33]:
def test_autoencoder(autoencoder, data_loader, device):
    autoencoder.eval()  # Переключение в режим оценки
    total_loss = 0
    criterion = nn.MSELoss()

    with torch.no_grad():
        for img_batch in data_loader:
            img = img_batch[0].to(device)  # Изменение здесь
            reconstructed = autoencoder(img)
            loss = criterion(reconstructed, img)
            total_loss += loss.item()

    average_loss = total_loss / len(data_loader)
    return average_loss

# Преобразование изображений с проливами в тензоры PyTorch и создание DataLoader
proliv_images_tensor = torch.tensor(proliv_images, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
proliv_loader = DataLoader(TensorDataset(proliv_images_tensor), batch_size=64, shuffle=False)

# Тестирование модели
proliv_loss = test_autoencoder(autoencoder, proliv_loader, device)
print(f"Average loss on proliv images: {proliv_loss}")


Average loss on proliv images: 0.005532826607426007


In [34]:
def calculate_mse(autoencoder, data_loader):
    mse_loss = nn.MSELoss(reduction='none')
    losses = []
    with torch.no_grad():
        for img in data_loader:
            img = img[0].to(device)
            reconstructed = autoencoder(img)
            loss = mse_loss(reconstructed, img).mean([1, 2, 3])  # Средняя ошибка по каждому изображению
            losses.extend(loss.cpu().numpy())
    return np.array(losses)

In [35]:
proliv_images = load_images_to_numpy('dataset/proliv')
proliv_images_tensor = torch.tensor(proliv_images, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0

In [36]:
proliv_loader = DataLoader(TensorDataset(proliv_images_tensor), batch_size=64, shuffle=False)

In [37]:
proliv_mse = calculate_mse(autoencoder, proliv_loader)

In [42]:
threshold = np.percentile(proliv_mse, 0.0020579725969582796)

In [43]:
test_images = load_images_to_numpy('dataset/test/imgs')
test_images_tensor = torch.tensor(test_images, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
test_loader = DataLoader(TensorDataset(test_images_tensor), batch_size=64, shuffle=False)

test_mse = calculate_mse(autoencoder, test_loader)
test_predictions = test_mse > threshold

In [45]:

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

# Функция для загрузки меток из файла аннотаций
def load_labels(label_file_path):
    labels = pd.read_csv(label_file_path, header=None, sep=' ')
    return labels.iloc[:, 1].values  # Предполагая, что метки находятся во втором столбце

# Загрузка меток
test_labels = load_labels('dataset/test/test_annotation.txt')  # Укажите правильный путь к файлу

# Получение предсказаний из ранее выполненного шага
test_predictions = test_mse > threshold

# Вычисление confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, test_predictions).ravel()

# Вычисление TPR и TNR
tpr = tp / (tp + fn)
tnr = tn / (tn + fp)

print(f"True Positive Rate: {tpr * 100:.2f}%")
print(f"True Negative Rate: {tnr * 100:.2f}%")


True Positive Rate: 91.38%
True Negative Rate: 95.13%


In [41]:
from sklearn.metrics import roc_curve, auc

def find_optimal_cutoff(test_labels, test_scores):
    fpr, tpr, thresholds = roc_curve(test_labels, test_scores)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold

# Вычисление оптимального порога
optimal_threshold = find_optimal_cutoff(test_labels, test_mse)
print(f"Оптимальный порог MSE: {optimal_threshold}")

# Применение оптимального порога для классификации
test_predictions_optimal = test_mse > optimal_threshold


Оптимальный порог MSE: 0.0020579725969582796
